# 

In [14]:
import bpy
import mathutils
import numpy as np

In [15]:
import geomlib_blender as gb

In [66]:
A = np.random.normal(0,20,(3,3))
cov = np.matmul(A.T,A)
np.sqrt(np.linalg.eig(cov)[0])

array([42.37482672, 12.95495666,  3.49277044])

In [67]:
# 3d ellipsoid
val,vec = np.linalg.eig(cov)

mat = np.eye(4)
mat[:3,:3] = vec*np.sqrt(val[np.newaxis,:])
mat = mathutils.Matrix(mat)

bpy.ops.mesh.primitive_ico_sphere_add(
    subdivisions=4,
    radius=1,
    enter_editmode=False,
    align="WORLD",
    location=(0,0,0),
    # scale=np.sqrt(val),
    scale = (1,1,1),
)
bpy.context.object.matrix_world = mat

In [68]:
# intersection with z=0 plane
cov_inv = np.linalg.inv(cov)
cov_sl = np.linalg.inv(cov_inv[:2,:2])

val_sl,vec_sl = np.linalg.eig(cov_sl)

mat_sl = np.eye(4)
mat_sl[:2,:2] = vec_sl*np.sqrt(val_sl[np.newaxis,:])
mat_sl = mathutils.Matrix(mat_sl)

bpy.ops.mesh.primitive_circle_add(
    enter_editmode=True,
    align="WORLD",
    location=(0,0,0),
    scale=(1,1,1),
)
bpy.ops.mesh.edge_face_add()
bpy.ops.object.editmode_toggle()
bpy.context.object.matrix_world = mat_sl

In [69]:
# projection into z=0 plane
val2,vec2 = np.linalg.eig(cov[:2,:2])

mat2 = np.eye(4)
mat2[:2,:2] = vec2*np.sqrt(val2[np.newaxis,:])
mat2 = mathutils.Matrix(mat2)

bpy.ops.mesh.primitive_circle_add(
    enter_editmode=True,
    align="WORLD",
    location=(0,0,0),
    scale=(1,1,1),
)
bpy.ops.mesh.edge_face_add()
bpy.ops.object.editmode_toggle()
bpy.context.object.matrix_world = mat2